Read dataset

In [3]:
import sys, os
import re
import pandas as pd
import itertools, nltk, string 

def read_file(file):
    f = open(file, 'r')

    pattern_title = '\[t\]'
    pattern_sentence = '(?<=##).+'
    pattern_aspect = '.+(?=##)'
#     pattern_aspect_anomali = '\[(u|p|s|cc|cs)\]'

    review = []
    count = 0
    for a in f:
        if re.search('##', a):
            sentence = re.findall(pattern_sentence, a)[0]
            aspect = re.findall(pattern_aspect, a)
#             anomali = re.findall(pattern_aspect_anomali, a)
#             if anomali:
#                 print(sentence, aspect)
#             else:
            if len(aspect) > 0:
                aspect = aspect[0]
            else:
                aspect = ''
            review.append((sentence, aspect))

    df = pd.DataFrame(columns=['review','target'])
    for r in review:
        df = df.append({'review': r[0], 'target': r[1]}, ignore_index=True)
        
    return df
        
positive_lexicon = []
negative_lexicon = []

def read_lexicon():
    global positive_lexicon;
    global negative_lexicon;
    
    with open(os.path.join(os.path.abspath('opinion-lexicon-English/') , 'positive-words.txt'), 'r') as file:
        line = file.readline();
        while ";" in line:
            line = file.readline();
         
        positive_lexicon = file.readlines()
    
    with open(os.path.join(os.path.abspath('opinion-lexicon-English/') , 'negative-words.txt'), 'r', encoding = "ISO-8859-1") as file:
        line = file.readline();
        while ";" in line:
            line = file.readline();
        
        negative_lexicon = file.readlines()
        
    positive_lexicon = list(map(lambda word: word.rstrip("\n\r"), positive_lexicon))
    negative_lexicon = list(map(lambda word: word.rstrip("\n\r"), negative_lexicon))
    
        
df = read_file('dataset/bing_liu/Nikon coolpix 4300.txt')
read_lexicon()

In [4]:
df

,review,target
0,this camera is perfect for an enthusiastic ama...,camera[+2]
1,"the pictures are razor-sharp , even in macro .","picture[+3], macro[+3]"
2,it is small enough to fit easily in a coat poc...,size[+2][u]
3,it is light enough to carry around all day wit...,weight[+1][u]
4,operating its many features is easy and often ...,feature[+2][u]
5,the manual does a fine job filling in any blan...,manual[+2]
6,"the auto-focus performs well , but i love havi...","auto focus[+2],scene mode[+2]"
7,comes with a 16 mb compact flash and one recha...,rechargable battery[+2]
8,"i bought a 256 mb cf and a second battery , so...",
9,i enthusiastically recommend this camera .,camera[+2]


In [5]:
from nltk.parse.corenlp import CoreNLPDependencyParser

parser = CoreNLPDependencyParser()
parse = next(parser.raw_parse("my name is khan"))

In [6]:
import requests
import re

def get_tregex(text):
    url = "http://localhost:9001/tregex"
    request_params = {"pattern": "S < (NP $ VP)"}
    r = requests.post(url, data=text, params=request_params, timeout=120)
    print(r)
    try:
        return r.json()['sentences'][0]
    except:
        return []

def sentence_from_tree(s):
    p_wh = r'(?<=WHADVP).*?(?=\))'
    p_wh2 = r'(?<=WHNP).*?(?=\))'
    pattern = r'(?<= )[a-zA-Z].*?(?=\))'
    replaced = s.replace('\r\n', '')
    wh = re.findall(p_wh, replaced)
    wh2 = re.findall(p_wh2, replaced)
    for x in wh:
        replaced = replaced.replace(x, '')
    for x in wh2:
        replaced = replaced.replace(x, '')
    res = ' '.join(re.findall(pattern, replaced))
    return res

def get_clauses(sentences):
    clauses = []
    res = get_tregex(sentences)
    length = len(res)
    for x in range(0, length):
        clauses.append(sentence_from_tree(res[str(x)]['match']))
    if length > 1:
        for x in range(0, length - 1):
            clauses[x] = clauses[x].replace(clauses[x+1], '')
    return clauses

In [ ]:
# adding new dependencies for DP 'nmod' and'advmod'
dep_DP = ['amod', 'prep', 'nsubj', 'csubj', 'xsubj', 'dobj', 'iobj']
conj_DP = ['conj']

In [ ]:
def rule_1_1(parse, f, o_expanded):
    #Rule 1.1
    for (w1, dep, w2) in list(parse.triples()):
        if(dep in dep_DP):
            # Rule 1.1
            if(w1[0] in o_expanded):
                if w2[1] == 'NN' and w2[0] not in f:
                    return True, w2[0]
            elif(w2[0] in o_expanded):
                if w1[1] == 'NN' and w1[0] not in f:
                    return True, w1[0]
    return False, None

def rule_1_2(parse, f, o_expanded):
    # Rule 1.2
    for (w1, dep, w2) in parse.triples():
        if(dep in dep_DP):
            H = ''
            O = ''
            if w1[0] in o_expanded:
                H = w2[0]
                O = w1
            elif w2[0] in o_expanded:
                H = w1[0]
                O = w2

            if H:
                for (w1, dep, w2) in list(parse.triples()):
                    if w1[0] == H and w2[0] != O[0]:
                        if w2[1] == 'NN' and w2[0] not in f:
                            return True, w2[0]
                    elif w2[0] == H  and w1[0] != O[0]:
                        if w1[1] == 'NN' and w1[0] not in f:
                            return True, w1[0]
    return False, None

def rule_4_1(parse, f, o_expanded):
    # Rule 4.1
    for (w1, dep, w2) in list(parse.triples()):
        if(dep in conj_DP):
            if w1[0] in o_expanded:
                if w2[1] == 'JJ' and w2[0] not in o_expanded:
                    return True, w2[0]

            elif w2[0] in o_expanded:
                if w1[1] == 'JJ' and w1[0] not in o_expanded:
                    return True, w1[0]

    return False, None

def rule_4_2(parse, f, o_expanded):
    # Rule 4.2
    for (w1, dep, w2) in parse.triples():
        if(dep in dep_DP or dep in conj_DP):
            H = ''
            O = ''
            if w1[0] in o_expanded:
                H = w2[0]
                O = w1
            elif w2[0] in o_expanded:
                H = w1[0]
                O = w2

            if H:
                for (w1, dep, w2) in list(parse.triples()):
                    if w1[0] == H and w2[0] != O[0]:
                        if w2[1] == 'JJ' and w2[0] not in o_expanded:
                            return True, w2[0]

                    elif w2[0] == H  and w1[0] != O[0]:
                        if w1[1] == 'JJ' and w1[0] not in o_expanded:
                            return True, w1[0]
    
    return False, None

def rule_3_1(parse, f, f_i):
    #Rule 3.1
    for (w1, dep, w2) in list(parse.triples()):
        if(dep in conj_DP):
            if(w1[0] in f_i): 
                if w2[1] == 'NN' and w2[0] not in f:
                    return True, w2[0]
            elif(w2[0] in f_i):          
                if w1[1] == 'NN' and w1[0] not in f:
                    return True, w1[0]
                
    return False, None

def rule_3_2(parse, f, f_i):
    # Rule 3.2
    for (w1, dep, w2) in parse.triples():
        if(dep in dep_DP or dep in conj_DP):
            H = ''
            O = ''
            if w1[0] in f_i:
                H = w2[0]
                O = w1
            elif w2[0] in f_i:
                H = w1[0]
                O = w2

            if H:
                for (w1, dep, w2) in list(parse.triples()):
                    if w1[0] == H and w2[0] != O[0]:
                        if w2[1] == 'NN' and w2[0] not in f:
                            return True, w2[0]
                    elif w2[0] == H  and w1[0] != O[0]:
                        if w1[1] == 'NN' and w1[0] not in f:
                            return True, w1[0]

    return False, None

def rule_2_1(parse, f, f_i, o_expanded):
    # Rule 2.1
    for (w1, dep, w2) in list(parse.triples()):
        if(dep in dep_DP):
            if w1[0] in f_i:
                if w2[1] == 'JJ' and w2[0] not in o_expanded:
                    return True, w2[0]

            elif w2[0] in f_i:
                if w1[1] == 'JJ' and w1[0] not in o_expanded:
                    return True, w1[0]
                
    return False, None

def rule_2_2(parse, f, f_i, o_expanded):      
    # Rule 2.2
    for (w1, dep, w2) in parse.triples():
        if(dep in dep_DP):
            H = ''
            O = ''
            if w1[0] in f_i:
                H = w2[0]
                O = w1
            elif w2[0] in f_i:
                H = w1[0]
                O = w2

            if H:
                for (w1, dep, w2) in list(parse.triples()):
                    if w1[0] == H and w2[0] != O[0]:
                        if w2[1] == 'JJ' and w2[0] not in o_expanded:    
                            return True, w2[0]
                    elif w2[0] == H  and w1[0] != O[0]:
                        if w1[1] == 'JJ' and w1[0] not in o_expanded:
                            return True, w1[0]

    return False, None

def clause(other_clause, temp, parse, f):
    if other_clause and len(temp) == 0:
        for (w1, dep, w2) in list(parse.triples()):
            if w1[1] == 'NN' and w1[0] not in f:
                return True, w1[0]
            
    return False, None

In [ ]:
import pandas as pd
candidate_aspect = []
new_opinion = []
op_set = positive_lexicon + negative_lexicon

def double_propagation(O: op_set, reviews, targets, output_file, save_to_file = False, using_clause = False):
    o_expanded = O
    f = []
    is_stop = False
    flag_cycle = 0
    
    t_a_p = []
    a_p = []
    r_p = []
    
    while (not is_stop):
        f_i = []
        o_i = []
    
        index = 0

        for sent in reviews:
            sentences = []
            if using_clause and flag_cycle == 0:
                clauses = get_clauses(sent)
                length = len(clauses)
                if length == 0:
                    sentences.append(sent)
                else:
                    for x in range(length - 1, -1, -1):
                        sentences.append(clauses[x])
            else:
                sentences.append(sent)
                
            other_clause = False            
            
            temp = []
#             if sentences:
#                 r = sentences
            for r in sentences:    
                
                try:
                    parse = next(parser.raw_parse(r))
                except:
                    continue

                        
                # Rule 1.1
                is_true, t_a = rule_1_1(parse, f, o_expanded)
                if is_true:
                    f_i.append(t_a)
                    temp.append(t_a)
                    
                # Rule 1.2
                is_true, t_a = rule_1_2(parse, f, o_expanded)
                if is_true:
                    f_i.append(t_a)
                    temp.append(t_a)
                
                # Rule 4.1
                is_true, t_o = rule_4_1(parse, f, o_expanded)
                if is_true:
                    o_i.append(t_o)
                
                # Rule 4.2
                is_true, t_o = rule_4_2(parse, f, o_expanded)
                if is_true:
                    o_i.append(t_o)

                    
                if using_clause and flag_cycle == 0:
                    flag_o = False
                    for word in r.split(' '):
                        if word in o_expanded:
                            flag_o = True
                            break

                    # clause
                    is_true, t_a = clause(other_clause, temp, parse, f)
                    if is_true:
                        f_i.append(t_a)
                        temp.append(t_a)

                    if flag_o and len(temp) == 0:
                        other_clause = True

                
            if flag_cycle == 0:
                r_p.append(sent)
                a_array = []
                tes = []
                try:
                    tes = targets[index].split(', ')
                except:
                    tes = []
                for x in tes:
                    a_array.append(x.split('[')[0])
            
                a_p.append('|'.join(a_array))
                t_a_p.append('|'.join(temp))
            else:
                if len(temp) != 0:
                    t_a_p[index] += '|' + '|'.join(temp)
            index += 1
            

        #calculate target and opinion expanded
        f = f + f_i 
        o_expanded = o_expanded + o_i

        
        #reread review, and run rule 3.1, 3.2, 2.1, and 2.2
        index = 0
        f_ii = []
        o_ii = []
        
        for sentences in reviews:
            
            temp = []
            if sentences:
                r = sentences
                
                try:
                    parse = next(parser.raw_parse(r))
                except:
                    continue

                flag_o = False
                for word in r.split(' '):
                    if word in o_expanded:
                        flag_o = True
                        break

                # Rule 3.1
                is_true, t_a = rule_3_1(parse, f, f_i)
                if is_true:
                    f_ii.append(t_a)
                    temp.append(t_a)
                    
                # Rule 3.2
                is_true, t_a = rule_3_2(parse, f, f_i)
                if is_true:
                    f_ii.append(t_a)
                    temp.append(t_a)
               

                # Rule 2.1
                is_true, t_o = rule_2_1(parse, f, f_i, o_expanded)
                if is_true:
                    o_ii.append(t_o)
                    
                # Rule 2.2
                is_true, t_o = rule_2_2(parse, f, f_i, o_expanded)
                if is_true:
                    o_ii.append(t_o)

            if len(temp) != 0:
                t_a_p[index] += '|' + '|'.join(temp)

            index += 1
            
        f_i = f_i + f_ii
        o_i = o_i + o_ii
        f = f + f_ii
        o_expanded = o_expanded + o_ii     
        
        flag_cycle = 1
        
        if(len(f_i) == 0 and len(o_i) == 0):
            if save_to_file == True:
                out = pd.DataFrame(r_p)
                out['aspect'] = a_p
                out['prediction'] = t_a_p
                out.to_csv(output_file)
            is_stop = True
        
    return f, o_expanded

In [ ]:
dp_aspect, opinion_expand = double_propagation(op_set, df['review'], df['target'], 'coba.csv', True, False)

In [ ]:
def calculate_frequency(aspects):
    aspect_frequency = {}
    
    for aspect in aspects:
        if(aspect in aspect_frequency):
            aspect_frequency[aspect] += 1
        else:
            aspect_frequency[aspect] = 1
            
    return aspect_frequency

k_DP = ['conj', 'compound']
def pruning_based_on_clause(aspect_frequency, reviews, predictions):
    pruning = []
    index = 0
    for review in reviews:
        temp = predictions[index]
        index += 1
        if isinstance(temp, str):
            prediction = temp.split('|')
        else:
            continue
        parse = next(parser.raw_parse(review))
        for (w1, dep, w2) in list(parse.triples()):
            if(w1[0] in aspect_frequency and w2[0] in aspect_frequency):
                if(dep not in k_DP):
                    if(aspect_frequency[w1[0]] > aspect_frequency[w2[0]]):
                        if w2[0] in prediction:
                            print(aspect_frequency[w1[0]], aspect_frequency[w2[0]], w2, prediction)
                            pruning.append(w2[0])
                    elif(aspect_frequency[w1[0]] < aspect_frequency[w2[0]]):
                        if w1[0] in prediction:
                            print(aspect_frequency[w1[0]], aspect_frequency[w2[0]], w1, prediction)
                            pruning.append(w1[0])
                
    return pruning
def pruning_based_other_products_and_dealers(aspect_frequency, reviews,predictions, window=3):
    pruning = []
    ProductINDI = ["compare to", "compare with", "better than", "worse than"]
    DealerINDI  = ["shop with", "buy from"]
    count = 0
    for review in reviews:
        temp = predictions[count]
        count += 1
        if isinstance(temp, str):
            prediction = temp.split('|')
        else:
            continue
        if any(indication in review for indication in ProductINDI):
            tokens = nltk.word_tokenize(review)
            index = 0
            while index < len(tokens) - 1:
                if tokens[index] + " " + tokens[index + 1] in ProductINDI:
                    index += 2
                    for x in range(window):
                        next_window = index + x + 1;
                        if next_window < len(tokens) and tokens[next_window] in aspect_frequency:
                            if tokens[next_window] in prediction:
#                                 print(review, tokens[next_window])
                                pruning.append(tokens[next_window])
                else :
                    index += 1
                    
        if any(indication in review for indication in DealerINDI):
            tokens = nltk.word_tokenize(review)
            index = 0
            while index < len(tokens) - 1:
                if tokens[index] + " " + tokens[index + 1] in DealerINDI:
                    index += 2
                    for x in range(window):
                        next_window = index + x + 1;
                        if next_window < len(tokens) and tokens[next_window] in aspect_frequency:
                            if tokens[next_window] in prediction:
#                                 print(review, tokens[next_window])
                                pruning.append(tokens[next_window])
                else :
                    index += 1 
                        
    return pruning


In [ ]:
import pandas as pd

hs = pd.read_csv("coba.csv")
# hs['prediction']

In [ ]:
import re
import math
import numpy as np
target = []

for t in hs['aspect']:
    if t is not np.nan:
        for s in t.split('|'):
            target.append(s)

In [ ]:
ap = calculate_frequency(dp_aspect)

In [ ]:
pruning_clause = pruning_based_on_clause(ap,hs['0'], hs['prediction']);
# print(pruning_clause)

In [ ]:
prun_dealer_product = pruning_based_other_products_and_dealers(ap, hs['0'], hs['prediction']);
prun_dealer_product

In [ ]:
ap

In [ ]:
a = pruning_clause + prun_dealer_product

for x in list(dict.fromkeys(dp_aspect)):
    if ap[x] == 1:
        ap[x] -= 1

for freq in list(dict.fromkeys(a)):
    try:
        if ap[freq] > 0:
            ap[freq] -= 1
    except: continue

In [ ]:
new_dp_aspect = []
for a in ap:
    if ap[a] > 0:
        new_dp_aspect.append(a)

In [ ]:
t_a = []
for a in hs['aspect']:
    if isinstance(a, str):
        for b in a.split('|'):
            for c in b.split(' '):
                t_a.append(c)
        

In [ ]:
tp = calculate_frequency(target)

In [ ]:
atas_recall = 0
atas_precision = 0
bawah_precision = 0
bawah_recall = 0

new_target = []
for x in list(dict.fromkeys(target)):
    for y in x.split(' '):
        new_target.append(y)

new_aspect = []
for x in list(dict.fromkeys(new_dp_aspect)):
    for y in x.split(' '):
        new_aspect.append(y)

for x in tp:
    bawah_precision += tp[x]
    for y in x.split(' '):
        if y in new_aspect:
            atas_precision += tp[x]
            break
            
for x in ap:
    bawah_recall += ap[x]
    for y in x.split(' '):
        if y in new_target:
            atas_recall += ap[x]
            break

precision = atas_precision/bawah_precision
recall = atas_recall/bawah_recall
f1 = (2 * precision * recall) / (precision + recall)
print(precision, recall, f1)

In [ ]:
get_tregex('')